In [2]:
# 분석 모듈
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime

# 크롤링 모듈
from selenium import webdriver
from bs4 import BeautifulSoup
import re

import os
os.chdir('C:/BigData/SooDa/2021/2021_과제_3')

In [6]:
# 크롬 드라이버 실행
driver = webdriver.Chrome('C:/Temp/chromedriver_win32/chromedriver.exe')

for i in range(3):
    
    url = 'http://www.statiz.co.kr/stat.php?mid=stat&re=0&ys=1982&ye=2020&un=2&sn=100&pa={}'.format(i*100)
    
    driver.get(url)
    driver.implicitly_wait(5)
    
    html = driver.find_element_by_xpath('//*[@id="mytable"]/tbody').get_attribute("innerHTML") # 기록 table을 str형태로 저장
    soup = BeautifulSoup(html, 'html.parser') # str 객체를 BeautifulSoup 객체로 변경
    
    temp = [i.text.strip() for i in soup.findAll("tr")] # tr 태그에서, text만 저장하기
    temp = pd.Series(temp) # list 객체에서 series 객체로 변경
    
    #'순'이나 'W'로 시작하는 row 제거
    # 즉, 선수별 기록만 남기고, index를 reset 해주기
    temp = temp[~temp.str.match("[순W]")].reset_index(drop=True) 
    
    temp = temp.apply(lambda x: pd.Series(x.split(' '))) # 띄어쓰기 기준으로 나눠서 dataframe으로 변경
    
    # 선수 팀 정보 이후 첫번째 기록과는 space 하나로 구분, 그 이후로는 space 두개로 구분이 되어 있음. 
    # 그래서 space 하나로 구분을 시키면, 빈 column들이 존재 하는데, 해당 column들 제거 
    temp = temp.replace('', np.nan).dropna(axis=1) 
    
    # WAR 정보가 들어간 column이 2개 있다. (index가 1인 column과, 제일 마지막 column)
    # 그 중에서 index가 1인 columm 제거 
    temp = temp.drop(1, axis=1)
    
    # 선수 이름 앞의 숫자 제거
    temp[0] = temp[0].str.replace("^\d+", '')

    # 선수들의 생일 정보가 담긴 tag들 가지고 오기
    birth = [i.find("a") for i in soup.findAll('tr') if 'birth' in i.find('a').attrs['href']]
    
    # tag내에서, 생일 날짜만 추출하기 
    p = re.compile("\d{4}\-\d{2}\-\d{2}")
    birth = [p.findall(i.attrs['href'])[0] for i in birth]
    
    # 생일 column 추가
    temp['생일'] = birth
    
    # page별 완성된 dataframe을 계속해서 result에 추가 시켜주기 
    if i == 0:
        result = temp
    else:
        result = result.append(temp)
        result = result.reset_index(drop=True)
        
    print(i, "완료")
        
# column 명 정보 저장        
columns = ['선수'] + [i.text for i in soup.findAll("tr")[0].findAll("th")][4:-3] + ['타율', '출루', '장타', 'OPS', 'wOBA', 'wRC+', 'WAR+', '생일']

# column 명 추가
result.columns = columns

# webdriver 종료
driver.close()

print("최종 완료")

# 결과값 저장해두기
result.to_csv('statiz_origin_foreigner_1982_2020.csv', encoding="utf-8-sig")

0 완료
1 완료
2 완료
최종 완료


In [11]:
# 이름 분리 해서, '이름' column 추가
data['이름'] = data['선수'].str.findall("^[ㄱ-힣]+").apply(lambda x: x[0])

# 기존 '선수' column 에 있던 이름 정보 제거
data['선수'] = data.apply(lambda x: x['선수'].replace(x['이름'], ''), axis=1)

# '시즌' column 생성
data['시즌'] = data['선수'].apply(lambda x: x[:2])

# 기존 '선수' column 에 있던 시즌 정보 제거
data['선수'] = data.apply(lambda x: x['선수'].replace(x['시즌'],''), axis=1)

# 년도를 4자리 숫자로 바꾸어주기 ex) 98 -> 1998, 15 -> 2015
data['시즌'] = data['시즌'].apply(lambda x: int("20"+ x) if int(x)<30 else int("19"+x))

# 포지션
position = ['1B', '2B', '3B', 'SS', 'C', 'RF', 'LF', 'CF', 'DH'] 

# 포지션 여부 확인
data['포지션여부'] = data['선수'].apply(lambda x : (np.isin(x[-2:], position) or np.isin(x[-1:], position))) 

# 포지션 분리
data['포지션'] = data['선수'].apply(lambda x: x[-2:] if x[-2:] in position else x[-1:])

# 포지션이 기록 되어 있지 않으면, 'M'값 넣기 
data.loc[data['포지션여부']==False, '포지션'] = 'M'

# 포지션이 N인 선수 삭제
posi = data[data['포지션']=='M'].index
data = data.drop(posi)

In [12]:
# 팀 & 나이 column 생성
data['팀'] = data.apply(lambda x: x['선수'].replace(x['포지션'],'') if type(x['포지션'])==str else x['선수'], axis=1)
data['나이'] = data.apply(lambda x: x['시즌'] - int(x['생일'][:4]) + 1, axis=1)

# 선수별 고유 아이디 부여
info = data[['이름', '생일']].apply(lambda x: tuple(x), axis=1)

player_id = {}
for i,j in enumerate(info.unique()):
    player_id[j] = i
    
data['ID'] = data.apply(lambda x: player_id[tuple([x['이름'], x['생일']])], axis=1)

In [13]:
# numeric data로 변환
float_columns = data.iloc[:,1:27].columns
data[float_columns] = data[float_columns].astype(float).fillna(0)

In [14]:
# 필요한 features 선택
cols = ['이름', '생일']
data = data[cols]

# 전처리 완료된 데이터프레임 저장
data.to_csv('foreigner_1982_2020.csv', encoding="utf-8-sig")